In [1]:
import requests
import pandas as pd
from datetime import datetime

# Some parameters
token_data = {'token': 'bJHNCRwspvRYRMOHApDLoAKBmLRNtdjY'} # token for datasets extraction

In [2]:
# No longer needed reprecated below
#x = pd.read_csv('https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=ASN00084027&startDate=2016-01-01&endDate=2016-01-02&boundingBox=90,-180,-90,180')
#https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-marine&dataTypes=WIND_DIR,WIND_SPEED&stations=AUCE&startDate=2016-01-01&endDate=2016-01-02&boundingBox=90,-180,-90,180
#https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&dataTypes=DP01,DP05,DP10,DSND,DSNW,DT00,DT32,DX32,DX70,DX90,SNOW,PRCP&stations=ASN00084027&startDate=1952-01-01&endDate=1970-12-31&includeAttributes=true&format=pdf
#dt = pd.read_json("../../../tzava/out.json")

#dt["name"]

# Dataset creation 

We want to create a dataset with monthly temperature measurements for several countries. 

## Create a list with all the countries

For the specific category of data, e.g. temperature, we export a list with the countries and their ids where measurements are available.

In [3]:
parameters = {'datasetid':'GSOM', 'locationcategoryid':'CNTRY', 'datacategoryid':'TEMP','limit':500}
x = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/locations?', params = parameters, 
                 headers=token_data)
print(x.url)

j = x.json()
j = j['results']

https://www.ncei.noaa.gov/cdo-web/api/v2/locations?datasetid=GSOM&locationcategoryid=CNTRY&datacategoryid=TEMP&limit=500


In [4]:
countries_table = pd.DataFrame(j)
countries_table[['name', 'id']]

,name,id
0,Antigua & Barbuda,FIPS:AC
1,United Arab Emirates,FIPS:AE
2,Afghanistan,FIPS:AF
3,Algeria,FIPS:AG
4,Azerbaijan,FIPS:AJ
...,...,...
191,Vietnam,FIPS:VM
192,Namibia,FIPS:WA
193,Swaziland,FIPS:WZ
194,Zambia,FIPS:ZA


In [5]:
# for each of the countries in the list export the temperature information
countries_select = ["France", "Greece"]
# Get the corresponding ids
countries_select_ids = list(countries_table.loc[countries_table['name'].isin(countries_select),'id'])

In [6]:
countries_select_ids

['FIPS:FR', 'FIPS:GR']

## Selection of the categories of features we want to include.
We check for the categories of variables that are available in the dataset that we want to query. We can then use these categories to include the features that we want to study in our dataset.

In [7]:
# Get the data categories that are available in the dataset
datacategories_parameters = {'datasetid':'GSOM', 'limit':500}
x = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/datacategories?', params = datacategories_parameters , 
                 headers=token_data)
print(x.status_code)
j = x.json()
j = j['results']

datacategories_table = pd.DataFrame(j)

200


In [8]:
print(datacategories_table)

              name    id
0         Computed  COMP
1      Evaporation  EVAP
2             Land  LAND
3    Precipitation  PRCP
4         Sunshine   SUN
5  Air Temperature  TEMP
6             Wind  WIND


For example, we want to use the measurements of temperature and precipitation and explore the relation between those two.

For every category, we want to get the list of variables available in the dataset.

In [9]:
# Sample code used for testing the variables in the dataset
variables_df = pd.DataFrame()
#categories = ['TEMP','PRCP']
categories = datacategories_table['id'].unique().tolist()
for category in categories :
    test_parameters = {'datasetid':'GSOM', 'datacategoryid':category,'limit':500}
    response = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?', params = test_parameters, 
                 headers=token_data)
    print(response.url)
    
    response_json = response.json()
    response_json = response_json['results']

    df = pd.DataFrame(response_json)
    df['datacategoryid']=category

    variables_df = pd.concat([variables_df,df], axis=0)

print(variables_df)

https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=COMP&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=EVAP&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=LAND&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=PRCP&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=SUN&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=TEMP&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=WIND&limit=500
       mindate     maxdate                                               name  \
0   1763-01-01  2023-07-01                                Cooling Degree Days   
1   1781-01-01  2023-07-01  Number of days with greater than or equal to 0...   
2   1781-01-01  2023-07-01  Number of days with greater than or equal to 1...   
3   1

So we have a table that holds the variables that are under the data categories that we are interested in and the description of those variables.

In [10]:
# We get a list with the category ids
datatype_ids = list(variables_df['id'].unique())

There is a limit of 1000 rows that we can query from the database. In order to export the full dataset we need to request batches of 1000 rows, with the next batch to start where the previous batch stopped. In order to identify this point, we need to identify the last date that the dataset describes and start the new export from that date. This way we avoid dealing with dates where the limit was reached before the measurements from all stations are exported. In the final dataset there will be duplicates for those dates and some stations measurements, but we can simply drop them.

In [21]:
#datatype_ids = ["HDSD","TAVG"]

dt = pd.DataFrame()
minDate = '2014-01-01'
maxDate = pd.Timestamp.today().strftime('%Y-%m-%d')

for id in countries_select_ids:
    print(id)

    for datatype in datatype_ids:
        print(datatype)
        # Initialize the date and bool params for each of the country
        startDate = minDate
        endDate = maxDate
        finished = False

        # The loop runs while we have not reached the final date or if there is no new data to export
        while ((startDate <= maxDate) & (finished == False)) :
            print(startDate)
            data_parameters = {'datasetid':'GSOM', 'locationcategoryid':'CNTRY', 'locationid':id, 'datatypeid':datatype, 'startdate':startDate, 'enddate':endDate, 'limit':1000}
            xi = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/data?', params = data_parameters, 
                    headers=token_data)
            while(xi.status_code == 503) :
                xi = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/data?', params = data_parameters, 
                    headers=token_data)

            print(xi.url)
            ji=xi.json()
            if (len(ji.keys())==0):
                dti = pd.DataFrame()
            else:
                ji = ji['results']
                dti = pd.DataFrame(ji)
                dti['countryid']=id
                dti['datatype']=datatype

            # Keep only the part of the dataset that is not yet in dt with an anti-join
            if ((dt.empty == False)& (dti.empty == False)) :
                dti = dti.merge(dt[['date', 'station', 'countryid', 'datatype']], on = ['date', 'station', 'countryid', 'datatype'], how = 'left', indicator = True)
                dti = dti[dti['_merge']!="both"]
                dti.drop('_merge', axis = 1, inplace = True)
            
            if  (dti.shape[0] == 0) :
                finished = True

            dt = pd.concat([dt,dti], axis=0)

            # Take the max date of the dataset in order to continue the loop
            if (sum(dt.loc[dt['countryid'] == id, 'datatype'].unique()==datatype)!=0):
                SD = max(dt.loc[(dt['countryid'] == id) & (dt['datatype'] == datatype), 'date'])
                startDate = datetime.strptime(SD, '%Y-%m-%dT%H:%M:%S')
                startDate = startDate.strftime('%Y-%m-%d')

FIPS:FR
CLDD
2014-01-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CLDD&startdate=2014-01-01&enddate=2023-09-04&limit=1000
2015-10-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CLDD&startdate=2015-10-01&enddate=2023-09-04&limit=1000
2017-07-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CLDD&startdate=2017-07-01&enddate=2023-09-04&limit=1000
2019-05-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CLDD&startdate=2019-05-01&enddate=2023-09-04&limit=1000
2022-07-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CLDD&startdate=2022-07-01&enddate=2023-09-04&limit=1000
2023-07-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datas

Check the shape and the types of the features that are available in the dataset.

In [22]:
dt.loc[dt['countryid'] == id, 'datatype'].unique()

array(['CLDD', 'DP01', 'DP10', 'DT00', 'DT32', 'DX32', 'DX70', 'DX90',
       'HTDD', 'EMXP', 'PRCP', 'CDSD', 'EMNT', 'EMXT', 'HDSD', 'TAVG',
       'TMAX', 'TMIN'], dtype=object)

In [23]:
dt.shape

(104555, 6)

In [24]:
max_dates_country = dt.groupby(['countryid'])[['date']].agg(max)
print(max_dates_country)

                          date
countryid                     
FIPS:FR    2023-07-01T00:00:00
FIPS:GR    2023-07-01T00:00:00


In [25]:
dt[dt['date'].isin(max_dates_country['date'].unique())].sort_values('station')

,date,datatype,station,attributes,value,countryid
141,2023-07-01T00:00:00,PRCP,GHCND:FG000081405,"1,,,S",193.5,FIPS:FR
141,2023-07-01T00:00:00,EMXP,GHCND:FG000081405,"1,,S,25,",30.0,FIPS:FR
141,2023-07-01T00:00:00,DP01,GHCND:FG000081405,"1,S",22.0,FIPS:FR
141,2023-07-01T00:00:00,DP10,GHCND:FG000081405,"1,S",13.0,FIPS:FR
142,2023-07-01T00:00:00,EMXP,GHCND:FP000091925,",,S,01,",23.9,FIPS:FR
...,...,...,...,...,...,...
213,2023-07-01T00:00:00,DP01,GHCND:MFM00067005,",S",9.0,FIPS:FR
214,2023-07-01T00:00:00,DP01,GHCND:SBM00071805,",S",26.0,FIPS:FR
214,2023-07-01T00:00:00,DP10,GHCND:SBM00071805,",S",5.0,FIPS:FR
214,2023-07-01T00:00:00,EMXP,GHCND:SBM00071805,",,S,29,",31.8,FIPS:FR


Depending on the dates we evaluate the number of data points.
12 data points per year (one for every month) means that we need to consider 120 points per decade. But there are many stations so they multiply with the number of stations.

In [26]:
pd.date_range('2023-07-01',periods=10,freq='M')

DatetimeIndex(['2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31',
               '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29',
               '2024-03-31', '2024-04-30'],
              dtype='datetime64[ns]', freq='M')

In [27]:
dt['datatype'].unique()


array(['CLDD', 'DP01', 'DP10', 'DT00', 'DT32', 'DX32', 'DX70', 'DX90',
       'HTDD', 'DSND', 'EMSD', 'EMXP', 'PRCP', 'CDSD', 'EMNT', 'EMXT',
       'HDSD', 'TAVG', 'TMAX', 'TMIN'], dtype=object)

We save the dataset to a csv file to start our EDA.

In [28]:
final_dt = dt.merge(variables_df[['id', 'datacategoryid', 'name']], left_on = 'datatype', right_on = 'id', how = 'left')

In [30]:
final_dt.to_csv('data.csv')